In [1]:
import pandas as pd
import numpy as np

# Renewable energy capacity
- solar and wind plant-level data from GEM

In [44]:
country_crosswalk = pd.read_excel("../../data/country_names.xlsx",sheet_name = "crosswalk")

### Solar capacity

In [11]:
all_solar = pd.read_excel("../../data/global_GEM/raw_data/Global-Solar-Power-Tracker-May-2023.xlsx",sheet_name="Data")
all_solar = all_solar.replace("Hong Kong","China")
all_solar

,Date Last Researched,Country,Project Name,Phase Name,Project Name in Local Language / Script,Other Name(s),Capacity (MW),Capacity Rating,Technology Type,Status,...,"Local area (taluk, county)","Major area (prefecture, district)",State/Province,Subregion,Region,GEM location ID,GEM phase ID,Other IDs (location),Other IDs (unit/phase),Wiki URL
0,2022-08-16,Algeria,Adrar Solar Plant,NaN,NaN,NaN,20.0,MWp/dc,PV,operating,...,Adrar District,NaN,Adrar,Northern Africa,Africa,L800008,G800008,WRI: WKS0068905,NaN,https://gem.wiki/Adrar_Solar_Plant
1,2022-08-16,Algeria,Ain Azel solar farm,NaN,محطة توليد الكهرباء بعين زادة,NaN,20.0,unknown,PV,operating,...,Ain Arnat,NaN,Sétif,Northern Africa,Africa,L800010,G800010,WRI: WKS0065285,NaN,https://gem.wiki/Ain_Azel_solar_farm
2,2022-08-16,Algeria,Ain El Ibel Solar,PV 1,محطة عين الابل للطاقة الشمسية ١,Ain Albel 1 solar plant,20.0,unknown,PV,operating,...,Aïn El Ibel District,NaN,Djelfa,Northern Africa,Africa,L800012,G800012,WRI: WKS0068916,NaN,https://gem.wiki/Ain_El_Ibel_Solar
3,2022-08-16,Algeria,Ain El Ibel Solar,PV 2,محطة عين الابل للطاقة الشمسية ٢,Ain Albel 2 solar plant,33.0,unknown,PV,operating,...,Aïn El Ibel District,NaN,Djelfa,Northern Africa,Africa,L800012,G800013,WRI: WKS0073479,NaN,https://gem.wiki/Ain_El_Ibel_Solar
4,2022-08-16,Algeria,Ain El Mehl Solar,NaN,محطة عين الملح للطاقة الشمسية,Ain El Melh Solar,20.0,unknown,PV,operating,...,Aïn El Melh District,NaN,M'Sila,Northern Africa,Africa,L800013,G800014,WRI: WKS0068915,NaN,https://gem.wiki/Ain_El_Mehl_Solar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15898,2021-07-23,Saudi Arabia,Ras Al-Khafji Desalination Solar Facility,NaN,محطة الطاقة الشمسية بالخفجي,Al Khafji solar plant,15.0,unknown,PV,operating,...,NaN,NaN,Eastern Province,Western Asia,Asia,L804128,G804720,WRI: WKS0063560,NaN,https://gem.wiki/Ras_Al-Khafji_Desalination_So...
15899,2022-11-10,Syria,Adra solar farm,1,محطة عدرا للطاقة الشمسية,NaN,10.0,unknown,PV,operating,...,NaN,NaN,Damascus,Western Asia,Asia,L804149,G804742,NaN,NaN,https://gem.wiki/Adra_solar_farm
15900,2022-11-10,Syria,Jandar solar farm,2,محطة جندر للطاقة الشمسية,Homs Jandar,14.0,unknown,PV,announced,...,NaN,NaN,Homs,Western Asia,Asia,L804126,G804718,NaN,NaN,https://gem.wiki/Jandar_solar_farm
15901,2022-08-31,United Arab Emirates,Masdar City solar farm,NaN,محطة مصدر للطاقة الشمسية,NaN,10.0,unknown,PV,operating,...,Abu Dhabi,NaN,Abu Dhabi Emirate,Western Asia,Asia,L804082,G804664,WRI: WRI1008740,NaN,https://gem.wiki/Masdar_City_solar_farm


In [16]:
all_solar.columns

Index(['Date Last Researched', 'Country', 'Project Name', 'Phase Name',
       'Project Name in Local Language / Script', 'Other Name(s)',
       'Capacity (MW)', 'Capacity Rating', 'Technology Type', 'Status',
       'Start year', 'Retired year', 'Operator',
       'Operator Name in Local Language / Script', 'Owner',
       'Owner Name in Local Language / Script', 'Latitude', 'Longitude',
       'Location accuracy', 'City', 'Local area (taluk, county)',
       'Major area (prefecture, district)', 'State/Province', 'Subregion',
       'Region', 'GEM location ID', 'GEM phase ID', 'Other IDs (location)',
       'Other IDs (unit/phase)', 'Wiki URL'],
      dtype='object')

In [51]:
# only select those operating or retired
all_solar_select = all_solar[all_solar["Status"].isin(["operating","retired"])]
# select those with a start year 
all_solar_select = all_solar_select[all_solar_select["Start year"].isna()==False]
# remove the retired plants without an end year
ret_rows = all_solar_select[all_solar_select["Status"]=="retired"]
op_rows = all_solar_select[all_solar_select["Status"]=="operating"]
ret_rows_drop = ret_rows[ret_rows["Retired year"].isna()]
all_solar_select = all_solar_select.drop(index=ret_rows_drop.index)
# change the retire year of operating plants to be 5000
all_solar_select.loc[op_rows.index,"Retired year"] = 5000
# and retired after current year
min_year = all_solar_select["Start year"].min()
for this_year in min_year+np.arange(2022-min_year+1):
    all_solar_year = all_solar_select[all_solar_select["Start year"]<=this_year]
    all_solar_year = all_solar_year[all_solar_year["Retired year"]>this_year]
    sum_year = all_solar_year.groupby("Country").sum()[["Capacity (MW)"]]
    sum_year["year"] = this_year
    if this_year == min_year+np.arange(2022-min_year+1)[0]:
        sum_all_years = sum_year
    else:
        sum_all_years = pd.concat([sum_all_years,sum_year])
# rename
sum_all_years = pd.merge(sum_all_years,country_crosswalk[["Country Code","GEM_Name"]],left_index=True,right_on="GEM_Name")
sum_all_years = sum_all_years.drop(columns="GEM_Name")
sum_all_years = sum_all_years.rename(columns={"Capacity (MW)":"Solar_MW"})
sum_all_years

,Solar_MW,year,Country Code
202,30.0,1985.0,USA
202,90.0,1986.0,USA
202,120.0,1987.0,USA
202,180.0,1988.0,USA
202,260.0,1989.0,USA
...,...,...,...
155,415.0,2022.0,POL
203,100.0,2021.0,UZB
203,200.0,2022.0,UZB
11,24.0,2022.0,AUT


In [59]:
# fill all values across countries and years
country_list = sum_all_years["Country Code"].unique()
sum_all_years_fn = sum_all_years.copy()
for this_year in min_year+np.arange(2022-min_year+1):
    this_year_data = sum_all_years.query("year==@this_year")
    this_year_countries = this_year_data["Country Code"]
    missing_list = list(set(country_list) - set(this_year_countries))
    missing_df = pd.DataFrame(data = {"Country Code":missing_list, "year":this_year, "Solar_MW":0})
    sum_all_years_fn = pd.concat([sum_all_years_fn,missing_df])
sum_all_years_fn = sum_all_years_fn.sort_values(by=["year","Country Code"]).set_index("year")
sum_all_years_fn

,Solar_MW,Country Code
year,,
1985.0,0.0,ARE
1985.0,0.0,ARG
1985.0,0.0,AUS
1985.0,0.0,AUT
1985.0,0.0,AZE
...,...,...
2022.0,200.0,UZB
2022.0,11835.0,VNM
2022.0,2925.6,ZAF


### Wind capacity

In [61]:
all_wind = pd.read_excel("../../data/global_GEM/raw_data/Global-Wind-Power-Tracker-May-2023.xlsx",sheet_name="Data")
all_wind = all_wind.replace("Hong Kong","China")
all_wind

,Date Last Researched,Country,Project Name,Phase Name,Project Name in Local Language / Script,Other Name(s),Capacity (MW),Installation Type,Status,Start year,...,"Local area (taluk, county)","Major area (prefecture, district)",State/Province,Subregion,Region,GEM location ID,GEM phase ID,Other IDs (location),Other IDs (unit/phase),Wiki URL
0,2022-11-10,Algeria,Kabertene wind farm,NaN,"مزرعة رياح كبيرت,, مدينة أدرار",Kabartene wind farm,10.0,onshore,operating,2014.0,...,Tsabit District,NaN,Adrar,Northern Africa,Africa,L900124,G900162,NaN,NaN,https://gem.wiki/Kabertene_wind_farm
1,2022-11-10,Algeria,Khenchela wind farm,NaN,مزرعة رياح خنشلة,NaN,20.0,onshore,cancelled,NaN,...,NaN,NaN,NaN,Northern Africa,Africa,L900137,G900178,NaN,NaN,https://gem.wiki/Khenchela_wind_farm
2,2022-11-10,Algeria,Timimoun wind farm,NaN,NaN,Timimoun Sktm,50.0,unknown,cancelled,NaN,...,NaN,NaN,NaN,Northern Africa,Africa,L916594,G920620,NaN,NaN,https://gem.wiki/Timimoun_wind_farm
3,2022-11-10,Algeria,Upper Plateaus wind farm,NaN,NaN,NaN,5010.0,unknown,pre-construction,2030.0,...,NaN,NaN,NaN,Northern Africa,Africa,L900099,G900128,NaN,NaN,https://gem.wiki/Upper_Plateaus_wind_farm
4,2022-08-25,Angola,Benjamin wind farm,NaN,NaN,NaN,52.0,onshore,pre-construction,2028.0,...,NaN,NaN,Benguela Province,Sub-Saharan Africa,Africa,L916595,G920621,NaN,NaN,https://gem.wiki/Benjamin_wind_farm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25186,2022-07-19,Uruguay,Valentines wind farm,NaN,Parque Eólico Valentines,NaN,70.0,onshore,operating,2016.0,...,Valentines,NaN,Treinta y Tres Department,Latin America and the Caribbean,Americas,L904995,G906199,NaN,NaN,https://gem.wiki/Valentines_wind_farm
25187,2022-07-19,Uruguay,Villa Rodriguez wind farm,NaN,Parque Eólico Villa Rodriguez,NaN,10.0,onshore,operating,2016.0,...,NaN,NaN,San José,Latin America and the Caribbean,Americas,L905509,G907540,NaN,NaN,https://gem.wiki/Villa_Rodriguez_wind_farm
25188,2022-07-19,Venezuela,La Guajira wind farm,1-A,Parque Eólico La Guajira,NaN,12.0,onshore,mothballed,2013.0,...,Municipio Indígena Guajira,NaN,Zulia State,Latin America and the Caribbean,Americas,L905443,G907342,NaN,NaN,https://gem.wiki/La_Guajira_wind_farm
25189,2022-07-19,Venezuela,La Guajira wind farm,1-B,Parque Eólico La Guajira,NaN,12.0,onshore,mothballed,2013.0,...,Municipio Indígena Guajira,NaN,Zulia State,Latin America and the Caribbean,Americas,L905443,G907343,NaN,NaN,https://gem.wiki/La_Guajira_wind_farm


In [62]:
all_wind.columns

Index(['Date Last Researched', 'Country', 'Project Name', 'Phase Name',
       'Project Name in Local Language / Script', 'Other Name(s)',
       'Capacity (MW)', 'Installation Type', 'Status', 'Start year',
       'Retired year', 'Operator', 'Operator Name in Local Language / Script',
       'Owner', 'Owner Name in Local Language / Script', 'Latitude',
       'Longitude', 'Location accuracy', 'City', 'Local area (taluk, county)',
       'Major area (prefecture, district)', 'State/Province', 'Subregion',
       'Region', 'GEM location ID', 'GEM phase ID', 'Other IDs (location)',
       'Other IDs (unit/phase)', 'Wiki URL'],
      dtype='object')

In [65]:
# only select those operating or retired
all_wind_select = all_wind[all_wind["Status"].isin(["operating","retired"])]
# select those with a start year 
all_wind_select = all_wind_select[all_wind_select["Start year"].isna()==False]
# remove the retired plants without an end year
ret_rows = all_wind_select[all_wind_select["Status"]=="retired"]
op_rows = all_wind_select[all_wind_select["Status"]=="operating"]
ret_rows_drop = ret_rows[ret_rows["Retired year"].isna()]
all_wind_select = all_wind_select.drop(index=ret_rows_drop.index)
# change the retire year of operating plants to be 5000
all_wind_select.loc[op_rows.index,"Retired year"] = 5000
# and retired after current year
min_year = all_wind_select["Start year"].min()
for this_year in min_year+np.arange(2022-min_year+1):
    all_wind_year = all_wind_select[all_wind_select["Start year"]<=this_year]
    all_wind_year = all_wind_year[all_wind_year["Retired year"]>this_year]
    sum_year = all_wind_year.groupby("Country").sum()[["Capacity (MW)"]]
    sum_year["year"] = this_year
    if this_year == min_year+np.arange(2022-min_year+1)[0]:
        sum_all_wind = sum_year
    else:
        sum_all_wind = pd.concat([sum_all_wind,sum_year])
# rename
sum_all_wind = pd.merge(sum_all_wind,country_crosswalk[["Country Code","GEM_Name"]],left_index=True,right_on="GEM_Name")
sum_all_wind = sum_all_wind.drop(columns="GEM_Name")
sum_all_wind = sum_all_wind.rename(columns={"Capacity (MW)":"Wind_MW"})
sum_all_wind

,Wind_MW,year,Country Code
63,23.0,1970.0,FRA
63,23.0,1971.0,FRA
63,23.0,1972.0,FRA
63,23.0,1973.0,FRA
63,23.0,1974.0,FRA
...,...,...,...
141,10.0,2021.0,NGA
141,10.0,2022.0,NGA
166,400.0,2021.0,SAU
166,400.0,2022.0,SAU


In [67]:
# fill all values across countries and years
country_list = sum_all_wind["Country Code"].unique()
sum_all_wind_fn = sum_all_wind.copy()
for this_year in min_year+np.arange(2022-min_year+1):
    this_year_data = sum_all_wind.query("year==@this_year")
    this_year_countries = this_year_data["Country Code"]
    missing_list = list(set(country_list) - set(this_year_countries))
    missing_df = pd.DataFrame(data = {"Country Code":missing_list, "year":this_year, "Wind_MW":0})
    sum_all_wind_fn = pd.concat([sum_all_wind_fn,missing_df])
sum_all_wind_fn = sum_all_wind_fn.sort_values(by=["year","Country Code"]).set_index("year")
sum_all_wind_fn

,Wind_MW,Country Code
year,,
1970.0,0.0,ABW
1970.0,0.0,ARG
1970.0,0.0,AUS
1970.0,0.0,AUT
1970.0,0.0,AZE
...,...,...
2022.0,1456.4,URY
2022.0,138173.0,USA
2022.0,3469.0,VNM


### Combine both

In [73]:
sum_wind = sum_all_wind_fn.reset_index().set_index(["year","Country Code"])
sum_solar = sum_all_years_fn.reset_index().set_index(["year","Country Code"])
main_RE = pd.merge(sum_wind,sum_solar,left_index=True,right_index=True,how="outer")
main_RE = main_RE.fillna(0)
main_RE["Wind_GW"] = main_RE["Wind_MW"]/1000
main_RE["Solar_GW"] = main_RE["Solar_MW"]/1000
main_RE.to_csv("_all_temporal_RE_capacity_GEM.csv")
main_RE

Wind_MW  Solar_MW  Wind_GW  Solar_GW
year   Country Code                                      
1970.0 ABW               0.0       0.0    0.000    0.0000
       ARG               0.0       0.0    0.000    0.0000
       AUS               0.0       0.0    0.000    0.0000
       AUT               0.0       0.0    0.000    0.0000
       AZE               0.0       0.0    0.000    0.0000
...                      ...       ...      ...       ...
2022.0 VNM            3469.0   11835.0    3.469   11.8350
       XKX             135.0       0.0    0.135    0.0000
       ZAF            3604.0    2925.6    3.604    2.9256
       ZMB               0.0      88.0    0.000    0.0880
       ZWE               0.0      25.0    0.000    0.0250

[5347 rows x 4 columns]